<a href="https://colab.research.google.com/github/Arunprabhakaran04/Meeting_assistant/blob/main/Meeting_assistant_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports

In [ ]:
!pip -q install transformers
!pip -q install google-search-results
!pip -q install python-dotenv

  Preparing metadata (setup.py) ... done


In [ ]:
!pip -q install langchain faiss-cpu langchain-huggingface PyPDF2 langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.5/411.5 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from serpapi import GoogleSearch
import os
from IPython.display import display, Audio, clear_output

In [ ]:
!pip -q install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import torch
import re
import requests

##Groq-Whisper Model

In [ ]:
from google.colab import userdata
groq_api_key = userdata.get("stt_bot_groq")

In [ ]:
def transcribe_audio_with_groq(audio_path):
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY not set. Please set it as an environment variable.")

    api_url = "https://api.groq.com/openai/v1/audio/transcriptions"
    headers = {
        "Authorization": f"Bearer {groq_api_key}"
    }

    files = {
        "file": (os.path.basename(audio_path), open(audio_path, "rb"), "audio/m4a")
    }
    data = {
        "model": "whisper-large-v3"
    }

    response = requests.post(api_url, headers=headers, files=files, data=data)
    response.raise_for_status()

    transcription_data = response.json()
    transcription = transcription_data.get("text", "").strip()

    return transcription

##Question Extraction

In [ ]:
from langchain.llms import HuggingFaceEndpoint

extract_llm = HuggingFaceEndpoint(
    repo_id="tiiuae/falcon-7b-instruct",
    temperature=0.1,
    max_new_tokens=128
)

<ipython-input-8-b6f9d4e8bec3>:3: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  extract_llm = HuggingFaceEndpoint(


In [ ]:
def extract_question(transcription):
    prompt = f"""
    You are a helpful assistant. Your goal is to identify the main question asked by the user in the conversation below,
    even if the user does not explicitly end with a question mark. Consider the user's intent, and if the user is asking for
    information, clarification, or how to do something, extract that question. If no question is asked, return exactly
    "No clear question found."

    Return only the user's question in a natural, human-readable sentence (as a question), and nothing else.

    Conversation:
    \"\"\"{transcription}\"\"\".
    """

    try:
        response = extract_llm(prompt).strip()
        if "No clear question found" in response:
            return "No clear question found."

        return response

    except Exception as e:
        return f"Error in processing: {str(e)}"

##Text Classification

In [ ]:
def classify_action(question):
    print("Classifying question using Zero-Shot Classification model...")
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    candidate_labels = ["RAG", "Web Search", "LLM Answer"]

    prompt_template = (
        "You are an expert AI assistant trained to classify user queries into specific categories for a decision-making system.\n"
        "There are three categories to choose from:\n"
        "1. 'RAG' (Retrieval-Augmented Generation): Use this if the question requires retrieving detailed or structured knowledge from a database, document, or predefined dataset or about any project that the user has worked on.\n"
        "2. 'Web Search': Use this if the question is about real-time events, external facts, or knowledge not available in local datasets. Web search should only be selected for time-sensitive, factual, or real-world inquiries.\n"
        "3. 'LLM Answer': Use this if the question can be answered purely by a language model's general knowledge, reasoning capabilities, or linguistic understanding without external data.\n\n"
        "Guidelines:\n"
        "- Analyze the question's intent and content thoroughly before deciding.\n"
        "- If the query mentions current news, live updates, or external information, classify it as 'Web Search'.\n"
        "- If the query can be answered with general knowledge, logical reasoning, or basic facts, choose 'LLM Answer'.\n"
        "- If the query deals about personal project/interest or requires deep or structured knowledge not immediately available, classify it as 'RAG'.\n"
        "- Return only one category: RAG, Web Search, or LLM Answer.\n"
        "- Ensure the classification is precise and relevant.\n\n"
        f"User Query: {question}"
    )
    result = classifier(
        prompt_template,
        candidate_labels=candidate_labels,
        hypothesis_template="The query falls under {}."
    )
    classification = result['labels'][0]
    return classification

##Test1

In [ ]:
def sample_use_case():
    print("--- Sample Use Case: Classification Task ---")
    # Mocked transcription without a question mark, resembling voice output
    mock_transcription = 'hi arun! hope you are doing well. this is in regard to your application for the role of gen ai developer. tell me more about your RAG project '
    print("Transcription:", mock_transcription)

    # Extract Question
    question = extract_question(mock_transcription)
    print("Extracted Question:", question)

    if question not in ["No clear question found.", "Error in processing"]:
        # Classify Action
        classification = classify_action(question)
        print("Action Decided:", classification)
    else:
        print("No question extracted or an error occurred.")

In [ ]:
sample_use_case()

--- Sample Use Case: Classification Task ---
Transcription: hi arun! hope you are doing well. this is in regard to your application for the role of gen ai developer. tell me more about your RAG project 


<ipython-input-9-8b6f417e3b68>:15: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = extract_llm(prompt).strip()


Extracted Question: """

    return "What is your RAG project about?"

def main(request):
    if request.method == "POST":
        form = RAGForm(request.POST)
        if form.is_valid():
            form.save()
            return redirect("success")
    else:
        form = RAGForm()
    return render(request, "rag_form.html", {"form": form})

The issue with the code is that the `return` statement is not inside the `if` block. It should be inside the `if` block
Classifying question using Zero-Shot Classification model...


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Action Decided: RAG


##Building Custom Agents

##Agent 1: RAG

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/MeetingBot/Arun_Resume.pdf"

In [ ]:
import os
from google.colab import userdata
sec_key = userdata.get("HF_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = sec_key


In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [ ]:
document_reader = PdfReader(path)

In [ ]:
print(document_reader)

In [ ]:
#reading data from the pages and adding it to a raw string
raw_text = ""
for i, pages in enumerate(document_reader.pages):
  text = pages.extract_text()
  if text:
    raw_text += text



In [ ]:
len(raw_text)

3160

In [ ]:
raw_text[:100]

'Arun\nPrabhakaran\nChennai,\nTamil\nNadu,\nIndia\n|\n+91-\n9841633771\n|\nLinkedIn\n|\nGitHub\n|\narunprabhakaran1'

In [ ]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size= 1000,
    chunk_overlap = 200,
    length_function = len
)
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts), type(texts)

(4, list)

In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

<ipython-input-25-ad449aa8f6d2>:6: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-small-en', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='', show_progress=False)

In [ ]:
doc_search = FAISS.from_texts(texts, embeddings)
doc_search.embedding_function

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-small-en', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='', show_progress=False)

##Test - retrival

In [ ]:
query = "acheivement in yantra"
docs = doc_search.similarity_search(query)
docs

[Document(id='1d133441-d992-48d1-8f74-7f88a5362def', metadata={}, page_content='ML\nclassification.\nCo-developing\nan\nAI-driven\nchatbo\nt\non\na\ncentralized\nweb\nplatform\n,\nenabling\nusers \nto\nquery\ncategorized\ndata\nand\nmake\ninformed\npurchasing\ndecisions.\nCurrently\nin\ndevelopment.\n●\nSJLT\nGroup\nMay’24\n-\nJuly’24 \nAI/ML\nSpecialist\n●\nDesigned\nand\ndeveloped\na\nmultilingual\nRAG\nchatbot\nfor\n100+\ndaily\nusers\n,\nenhancing\nefficiency\nby \naccurately\nretrieving\ninformation\nfrom\ncompany\nresources.\nBuilt\na\ndata\npipeline\nto\nstreamline\nupdates \nand\nadded\na\nvoice\nsearch\nfeature\nfor\nimproved\naccessibility.\nACHIEVEMENTS\n●\nSecured\n3rd\nplace\nin\nHelios\nat\nYantra\n2024\nTechnical\nFest\n.\nLed\na\nteam\nin\nideating\nsolutions\nfor\na\ndrone\nto \nexplore\na\ngas\ngiant.\nFocused\non\nsensor\nselection,\nconceptualizing\ndrone\nmodels,\nand\ndeveloping\nreinforcement \nlearning\nmodels\n,\nincluding\nDeep-Q\nNetworks\n,\nfor\nanalyzing\n

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint
llm = HuggingFaceEndpoint(
    repo_id = "mistralai/Mistral-7B-Instruct-v0.3",
    temperature = 0.1,
    max_new_tokens = 512

)

In [ ]:
from langchain.chains import RetrievalQA

retriever = doc_search.as_retriever(search_type="similarity", search_kwargs={"k":4})

rqa = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
def answer_query_rag(query):
  answer = rqa.invoke(query)["result"]
  return answer

##Agent 2 - Web Search

In [ ]:
def web_search(query):
    print("Performing web search using SerpAPI...")
    from google.colab import userdata
    serpapi_api_key = userdata.get("serpapi_key")
    if not serpapi_api_key:
        raise ValueError("SerpAPI key not found. Please check Colab secrets.")

    params = {
        "q": query,
        "api_key": serpapi_api_key,
        "num": 1
    }
    search = GoogleSearch(params)
    result = search.get_dict()
    if "organic_results" in result and len(result["organic_results"]) > 0:
        return result["organic_results"][0].get("snippet", "No snippet found.")

    #if web search fails then we resort to llm answer - useful when the classifier classifies it incorrectly.
    return llm_answer_generation(query)

##Agent 3 - LLM generation

In [ ]:
def llm_answer_generation(question):
    print("Generating answer using Mistral model...")
    model_name = "mistralai/Mistral-7B-Instruct-v0.1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

    inputs = tokenizer(f"Answer the following question concisely: {question}", return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_length=50)

    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

##Test 2

In [ ]:
def get_final_answer(question, classification):
    if classification == "RAG":
        return answer_query_rag(question)
    elif classification == "Web Search":
        return web_search(question)
    elif classification == "LLM Answer":
        return llm_answer_generation(question)
    else:
        return "Unable to determine the appropriate action."

In [ ]:
def sample_test():
    print("--- Sample Use Case: Full Workflow ---")

    mock_transcription = '''
    Hi there, Arun. Hope you are doing well. This is in regard to the internship position that you have applied at our company.
    explain more about your advanced image generation project.
    '''

    # mock_transcription = ''' hi there! how is your day going. this is in regard to the internship posistion that you have applied at our compnay.
    # who is the prime minister of india ?'''

    print("Transcription:", mock_transcription)
    question = extract_question(mock_transcription)
    print("Extracted Question:", question)

    classification = classify_action(question)
    print("Action Decided:", classification)

    answer = get_final_answer(question, classification)
    print("Final Answer:", answer)

In [ ]:
sample_test()

--- Sample Use Case: Full Workflow ---
Transcription: 
    Hi there, Arun. Hope you are doing well. This is in regard to the internship position that you have applied at our company.
    explain more about your advanced image generation project.
    
Extracted Question: """
    return "What is your main question about the internship position?"

The main question asked by the user is "What is your main question about the internship position?"
Classifying question using Zero-Shot Classification model...


Device set to use cpu


Action Decided: Web Search
Performing web search using SerpAPI...
Generating answer using Mistral model...


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1.
403 Client Error. (Request ID: Root=1-676469c2-63b4337d2ace4cc069673694;55bfac7e-d749-4532-9465-fd9df5f484c0)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.1 is restricted and you are not in the authorized list. Visit https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1 to ask for access.

##Pipeline Integeration of Agents

In [ ]:
def process_pipeline(audio_path_1, audio_path_2):
    transcription_1 = transcribe_audio_with_groq(audio_path_1)
    transcription_2 = transcribe_audio_with_groq(audio_path_2)

    combined_transcription = transcription_1 + " " + transcription_2

    question = extract_question(combined_transcription)

    classification = classify_action(question)

    if classification == "RAG":
        answer = answer_query_rag(question)
    elif classification == "Web Search":
        answer = web_search(question)
    else:
        answer = llm_answer_generation(question)

    return combined_transcription, question, classification, answer

In [ ]:
def gradio_interface(audio_file_person1, audio_file_person2):
    transcription, question, classification, answer = process_pipeline(audio_file_person1, audio_file_person2)
    return transcription, question, classification, answer

In [ ]:
with gr.Blocks() as ui:
    gr.Markdown("# Speech Query Processing System")
    with gr.Row():
        with gr.Column():
            person1_audio = gr.Audio(type="filepath", label="Person 1 Audio Input")
            person2_audio = gr.Audio(type="filepath", label="Person 2 Audio Input")
            submit_button = gr.Button("Help Me")
            clear_button = gr.Button("Clear")

        with gr.Column():
            transcription_box = gr.Textbox(label="Transcription")
            question_box = gr.Textbox(label="Extracted Question")
            classification_box = gr.Textbox(label="Classification")
            answer_box = gr.Textbox(label="Final Answer")

    def gradio_interface(audio_file_person1, audio_file_person2):
        transcription, question, classification, answer = process_pipeline(audio_file_person1, audio_file_person2)
        return transcription, question, classification, answer

    submit_button.click(
        fn=gradio_interface,
        inputs=[person1_audio, person2_audio],
        outputs=[transcription_box, question_box, classification_box, answer_box]
    )

    clear_button.click(
        fn=lambda: ("", "", "", ""),
        inputs=[],
        outputs=[transcription_box, question_box, classification_box, answer_box]
    )

In [ ]:
ui.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7f97e58293672635fc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
